In this notebook we perform the different classification tasks to determine which one is the best.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Metrics for Performance Evaluation

In [ ]:
class Performance_metrics():

    def __init__(self, y_actual, y_predicted):

        self.y_actual = y_actual # df of actual class labels
        self.y_predicted = y_predicted # df of predicted class labels

        self.df_confusion = pd.crosstab(self.y_actual, self.y_predicted)
        self.tp = self.df_confusion[1][1]
        self.tn = self.df_confusion[0][0]
        self.fp = self.df_confusion[0][1]
        self.fn = self.df_confusion[1][0]
        

    def cost_matrix(self, ctp, ctn, cfp, cfn):

        """Cost of classifying instances of class j as class i.
        Args:
        ctp: cost of true positive"
        ctn: cost of true negative""
        cfp: cost of false positive""
        cfn: cost of false negative""
        """

        self.ctp = ctp
        self.ctn = ctn
        self.cfp = cfp
        self.cfn = cfn

        cost_matrix = np.array([[self.ctn, self.cfp], [self.cfn, self.ctp]])
        return cost_matrix
    
    def accuracy(self):

        """Accuracy of the model.
        """
        accuracy = (self.tp + self.tn) / (self.tp + self.tn + self.fp + self.fn)
        return accuracy
    
    def cost(self):

        """Cost of the model.
        """
        cost = (self.tp * self.ctp) + (self.tn * self.ctn) + (self.fp * self.cfp) + (self.fn * self.cfn)
        return cost
    
    def precision(self):
            
        """Precision of the model.
        """
        precision = self.tp / (self.tp + self.fp)
        return precision
    
    def recall(self):
        
        """Recall of the model.
        """
        recall = self.tp / (self.tp + self.fn)
        return recall
    
    def f_measure(self):
    
        """F-measure of the model.
        """

        f_measure = 2*self.tp / (2*self.tp + self.fp + self.fn)
        return f_measure
    

# Feature Selection